<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/knu_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# word -> pdf 파일로 변환(로컬)
# Excel -> openpyxl로 텍스트 추출 후 저장
# HWP -> PDF 파일로 변환(로컬)
# ppt -> PDF 파일로 변환(로컬)
# 모든 html 페이지는 md로 저장
# jpg, pdf(사진) -> ocr로 텍스트 추출 후 저장

"""
앞으로 할일
# 워드 파일 -> pdf 파일로 변환 예시(완료)
# 엑셀 파일 -> pdf 변환 해야함(완료)
# pdf(사진) -> pdfplumber로도 추출이 안되면 사진이니 ocr 진행하기(완료)
# jpg, png 사진파일 -> 텍스트 추출 후 저장 (완료)
전처리 + 문서 청킹(나누기) 방법 (진행 중)

→ 학과는 너무 작게 나눔, 캠퍼스별로 collection을 나누고, 질문 전에 해당 collection만 검색.
사용자 질문 -> 잘 변환 -> 해당되는 collection을 불러와 안에 청크들을 조회하며 유사도 높은걸 검색한다.
"""

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요 라이브러리 설치
!pip install chromadb
!pip install sentence-transformers
!pip install pandas
!pip install PyPDF2
!pip install pdfplumber

# 엑셀 텍스트 추출
!pip install openpyxl

# ocr PaddleOCR 사용
!pip install paddleocr
!pip install "paddlepaddle==2.6.2" -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!apt-get install -y poppler-utils
!pip install pdf2image
!pip install matplotlib

In [ ]:
"""
# transformers 버전이 꼬이면 오류나면 다시 설치
# 1. transformers 삭제
!pip uninstall -y transformers

# 2. 안정적인 버전으로 재설치 (4.40.2 또는 4.41.1 권장)
!pip install transformers==4.41.2
"""

In [ ]:
# LangChain 아님
# 구글 드라이브 document_files에 있는 파일들 전부 저장
# 경고 및 오류 처리 방법
# incorrect startxref pointer(1) : Xref Table (객체 위치 정보) 를 찾기 위한 포인터가 잘못되었거나 손상된 경우 나타나는 경고, 되긴하나 일부 페이지나 객체 누락될 수 있음 -> 해결책 pdfplumber
# Advanced encoding /KSCms-UHC-H not implemented yet : DF가 한글 문자셋(KSCms-UHC-H) 을 사용하는데, PyPDF2 라이브러리는 이 고급 인코딩을 완벽하게 지원하지 않는다는 경고 -> 해결책 pdfplumber
# ⚠️ 2020-1 수강신청 안내문(컴퓨터공학전공).pdf: 텍스트 없음 → 스킵 pdf가 사진으로 되어 있는 경우 발생하는 문제 -> 해결책 ocr
import io, os, re, unicodedata, pytz, warnings, logging
import pandas as pd
import numpy as np
import pdfplumber
from datetime import datetime
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from chromadb import PersistentClient
from paddleocr import PaddleOCR
from pdf2image import convert_from_path
from difflib import get_close_matches, SequenceMatcher

warnings.filterwarnings("ignore", category=UserWarning, module='pdfminer')
logging.getLogger("pdfminer").setLevel(logging.ERROR)

# 설정
PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
campus_name = "cheonan"  # 또는 "gongju", "yesan", cheonan
COLLECTION_NAME = f"knu_{campus_name}_collection"
# 과거: COLLECTION_NAME = "knu_collection" 현재 : 캠퍼스별로 컬렉션 생성
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"
ROOT_FOLDER = "/content/drive/MyDrive/document files Cheonan Campus"     # 여기만 바꾸기(경로)
URL_MAPPING_SUFFIX = "_url.xlsx"

# PaddleOCR 초기화 (최초 1회만)
ocr = PaddleOCR(use_angle_cls=True, lang='korean')

KST = pytz.timezone('Asia/Seoul')
model = SentenceTransformer(EMBEDDING_MODEL_NAME)
client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)
existing_ids = set(collection.get(limit=None)['ids'])

# 텍스트 전처리 함수
def clean_text(text):
    # 공백, 특수문자 정리 + 너무 짧은 줄 제거
    text = unicodedata.normalize("NFC", str(text))
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[■◆●※▶▷▲→◇]', '', text)  # 불필요 기호 제거
    lines = [line.strip() for line in text.split('\n') if len(line.strip()) > 10]
    return '\n'.join(lines)

# 청킹 전략
def split_chunks(text, max_length=500, overlap=100):
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    all_chunks = []

    for para in paragraphs:
        # 문장 단위 분리
        sentences = re.split(r'(?<=[.!?])\s+', para)
        current_chunk = ""

        for sentence in sentences:
            if len(current_chunk) + len(sentence) <= max_length:
                current_chunk += sentence + " "
            else:
                all_chunks.append(current_chunk.strip())
                current_chunk = current_chunk[-overlap:] + sentence + " "

        if current_chunk.strip():
            all_chunks.append(current_chunk.strip())

    return [chunk for chunk in all_chunks if len(chunk) > 30]

# ✅ 파일명 정규화 함수
def normalize_filename(name: str) -> str:
    # 1. NFC 정규화 (한글 자소 분리 방지)
    name = unicodedata.normalize("NFC", str(name))
    # 2. 공백 양쪽 제거
    name = name.strip()
    # 3. 괄호, 따옴표, 하이픈 포함한 다양한 특수문자 제거
    name = re.sub(r"[\\/*?\"<>|‛‘’“”'「」【】·~!@#$%^&+=;,…ㆍ—−‒–―•※→←↑↓★☆♥♡•′″\[\]\(\)\{\},\-]", '', name)
    return name

# ✅ 유사 파일명을 통한 URL 매핑 함수
def find_best_url(base_name_norm, url_mapping, cutoff=0.7):
    # 유사도 100%
    if base_name_norm in url_mapping:
        return url_mapping[base_name_norm]

    # 이후 파일명 부일치시 유사도 기반 매칭
    # 모든 후보에 대해 유사도 계산
    candidates = []
    for key in url_mapping.keys():
        sim = SequenceMatcher(None, base_name_norm, key).ratio()
        candidates.append((key, sim))

    # 가장 높은 유사도 항목 추출
    best_match = max(candidates, key=lambda x: x[1])
    match_name, best_score = best_match

    if best_score >= cutoff:
        print(f"⚠️ 파일명 불일치 → 유사도 기반 매칭 사용: '{base_name_norm}' → '{match_name}' (유사도: {best_score:.2f})")
        return url_mapping[match_name]
    else:
        print(f"⛔ 파일명 매칭 실패: '{base_name_norm}' → 가장 비슷한 항목: '{match_name}' (유사도: {best_score:.2f})")
        return "출처 URL 없음"

# OCR 함수(사진 파일 .jpg, .jpeg, .png)
def extract_text_from_image(image_path):
    try:
        from PIL import Image
        img = Image.open(image_path).convert("RGB")
        image_np = np.array(img)[:, :, ::-1]  # RGB → BGR
        result = ocr.ocr(image_np, cls=True)
        text = "\n".join([line[1][0] for line in result[0]])
        if text.strip():
            print(f"🖼️ {os.path.basename(image_path)}: 이미지 OCR 추출 성공")
        return text
    except Exception as e:
        print(f"❌ [이미지 OCR 실패] {image_path}: {e}")
        return ""

# OCR 함수(pdf에 텍스트말고 사진이 저장된 경우)
def extract_text_with_paddleocr(pdf_path):
    try:
        images = convert_from_path(pdf_path, dpi=300, poppler_path="/usr/bin")
        full_text = ""
        for i, page in enumerate(images):
            image_np = np.array(page)[:, :, ::-1]  # RGB -> BGR
            result = ocr.ocr(image_np, cls=True)
            for line in result[0]:
                full_text += line[1][0] + "\n"
        return full_text
    except Exception as e:
        print(f"❌ [OCR 실패] {os.path.basename(pdf_path)}: {e}")
        return ""

# ✅ PDF 텍스트 추출 함수 (PyPDF2 -> pdfplumber -> ocr 순으로 모든 pdf 파일 텍스트 추출)
def read_file(filepath):
    def try_pypdf2(path):
        class SkipHandler(logging.Handler):
            def emit(self, record):
                if "incorrect startxref pointer" in record.getMessage():
                    raise ValueError("skip_due_to_incorrect_startxref")

        handler = SkipHandler()
        logger = logging.getLogger("PyPDF2._reader")
        logger.addHandler(handler)
        logger.setLevel(logging.WARNING)

        try:
            with warnings.catch_warnings(record=True) as w:
                warnings.simplefilter("always")
                reader = PdfReader(path)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() or ""
                for warning in w:
                    msg = str(warning.message)
                    if "KSCms-UHC-H" in msg:
                        print(f"⚠️ {os.path.basename(path)}: 한글 글꼴 미지원 (KSCms-UHC-H) → pdfplumber 시도")
                        return ""
                return text
        except ValueError as ve:
            if "skip_due_to_incorrect_startxref" in str(ve):
                print(f"⚠️ {os.path.basename(path)}: 구조 손상 (startxref) → pdfplumber 시도")
                return ""
            else:
                raise
        except Exception as e:
            print(f"❌ [PyPDF2 실패] {path}: {e}")
            return ""
        finally:
            logger.removeHandler(handler)

    def try_pdfplumber(path):
        try:
            import pdfplumber
            with pdfplumber.open(path) as pdf:
                text = ""
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
                if text.strip():
                    print(f"📄 {os.path.basename(path)}: pdfplumber 추출 성공")
                return text
        except Exception as e:
            print(f"❌ [pdfplumber 실패] {path}: {e}")
            return ""

    if filepath.endswith(".pdf"):
        text = try_pypdf2(filepath)
        if not text.strip():
            text = try_pdfplumber(filepath)
        return text

    elif filepath.endswith(".md"):
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                return f.read()
        except:
            return ""

    elif filepath.lower().endswith(('.jpg', '.jpeg', '.png')):
        return extract_text_from_image(filepath)

    elif filepath.lower().endswith(('.xlsx', '.xls')):
        try:
            xls = pd.read_excel(filepath, sheet_name=None, dtype=str)
            all_text = ""
            for sheet_name, df in xls.items():
                df = df.fillna("")
                all_text += f"\n=== 시트: {sheet_name} ===\n"
                all_text += "\n".join(["\t".join(row) for row in df.values.tolist()]) + "\n"
            return all_text.strip()
        except Exception as e:
            print(f"❌ 엑셀 읽기 실패: {filepath} ({e})")
            return ""

    return ""

def get_all_files(folder):
    all_files = []
    for dirpath, _, filenames in os.walk(folder):
        for f in filenames:
            if f.endswith(('.pdf', '.md', '.jpg', '.jpeg', '.png', '.xlsx', '.xls')):
                all_files.append(os.path.join(dirpath, f))
    return all_files

def extract_category(file_path, base_folder):
    rel_path = os.path.relpath(file_path, start=base_folder)
    parts = rel_path.split(os.sep)
    return parts[0] if len(parts) > 1 else "기타"

def build_metadata(file_path, base_folder, base_name, source_url, ext):
    category = extract_category(file_path, base_folder)
    department_path = os.path.relpath(base_folder, ROOT_FOLDER)

    # 확장자 기반으로 source_type 분기
    if ext == ".pdf":
        source_type = "pdf"
    elif ext == ".md":
        source_type = "markdown"
    elif ext in [".jpg", ".jpeg", ".png"]:
        source_type = "image"
    elif ext in [".xlsx", ".xls"]:
        source_type = "excel"
    else:
        source_type = "etc"  # 혹시 모를 확장자에 대한 대비

    return {
        "file_name": base_name,
        "department": department_path,
        "category": category,
        "source_type": source_type,
        "source_url": source_url,
        "date": datetime.now(KST).isoformat()
    }

total_added = 0
failed_matches = []

# 엑셀 기준으로 저장
for dirpath, _, filenames in os.walk(ROOT_FOLDER):
    for filename in filenames:
        if filename.endswith(URL_MAPPING_SUFFIX):
            folder = dirpath  # 엑셀 위치 폴더 = 기준 폴더
            dept_folder_name = os.path.basename(folder)
            url_file = os.path.join(folder, filename)

            try:
                url_df = pd.read_excel(url_file)
                url_df.columns = url_df.columns.str.strip()
                url_df['파일명_정규화'] = url_df['파일명'].apply(lambda x: normalize_filename(os.path.splitext(str(x))[0]))
                url_df['URL'] = url_df['URL'].astype(str).str.strip()
                url_mapping = dict(zip(url_df['파일명_정규화'], url_df['URL']))
            except Exception as e:
                print(f"❌ URL 매핑 실패: {url_file} ({e})")
                continue

            file_paths = get_all_files(folder)

            for file_path in file_paths:
                file_name = os.path.basename(file_path)
                base_name = os.path.splitext(file_name)[0]
                ext = os.path.splitext(file_name)[1].lower()

                if file_name.endswith(URL_MAPPING_SUFFIX):
                    continue

                # 텍스트 추출 결과가 공백이거나 빈 문자열일 경우 해당 pdf 파일은 사진으로 구성된 경우임
                raw_text = read_file(file_path)
                if not raw_text.strip():
                    print(f"⚠️ {file_name}: 텍스트 없음 → PaddleOCR 시도")
                    raw_text = extract_text_with_paddleocr(file_path)

                # 그래도 텍스트 없으면 최종 스킵
                if not raw_text.strip():
                    print(f"⛔ {file_name}: PyPDF2, pdfplumber, PaddleOCR 방법으로 텍스트 추출 실패 → 스킵")
                    continue

                # ✅ URL 매핑 실패하면 바로 전체 스킵
                norm_base = normalize_filename(base_name)
                source_url = find_best_url(norm_base, url_mapping)
                if source_url == "출처 URL 없음":
                    failed_matches.append((base_name, dept_folder_name))
                    print(f"⛔ {file_name}: URL 매칭 실패 (→ {norm_base}) → 벡터 저장 스킵")
                    continue

                cleaned = clean_text(raw_text) # 전처리 전략 함수 호출
                chunks = split_chunks(cleaned, max_length=500, overlap=100) # 청킹 전략 함수 호출
                embeddings = model.encode(chunks).tolist()
                ids = [f"{dept_folder_name}_{base_name}_chunk_{i}" for i in range(len(chunks))]

                new_chunks, new_embeddings, new_ids, new_metadatas = [], [], [], []

                for chunk, emb, id_ in zip(chunks, embeddings, ids):
                    if id_ not in existing_ids:
                        meta = build_metadata(file_path, folder, base_name, source_url, ext)
                        new_chunks.append(chunk)
                        new_embeddings.append(emb)
                        new_ids.append(id_)
                        new_metadatas.append(meta)

                if new_chunks:
                    collection.add(
                        documents=new_chunks,
                        embeddings=new_embeddings,
                        metadatas=new_metadatas,
                        ids=new_ids
                    )
                    print(f"✅ {file_name}: {len(new_chunks)}개 저장 완료")
                    total_added += len(new_chunks)
                else:
                    print(f"🟦 {file_name}: 중복 청크 존재 → 스킵")


if failed_matches:
    print(f"\nURL 매핑 실패 파일 수: {len(failed_matches)}")
    for base_name, dept in failed_matches:
        print(f"📂 [{dept}] - {base_name}")
else:
    print("\n✅ 모든 파일이 URL 매핑에 성공했습니다.")

print(f"\n🎉 총 저장된 신규 청크 수: {total_added}")

In [64]:
import random
from chromadb import PersistentClient

# 설정
PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_cheonan_collection"

client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

# 전체 문서 불러오기
results = collection.get(limit=None, include=["documents", "metadatas", "embeddings"])
total_docs = len(results['ids'])

print(f"\n📦 전체 저장된 벡터 수: {total_docs}")
print("=" * 70)

# 10개 무작위 추출
sample_indices = random.sample(range(total_docs), min(10, total_docs))

for i in sample_indices:
    doc_id = results["ids"][i]
    metadata = results["metadatas"][i]
    doc_text = results["documents"][i].strip().replace("\n", " ")
    embedding = results["embeddings"][i]

    print(f"🔹 ID: {doc_id}")
    print(f"   📁 파일명: {metadata.get('file_name')}")
    print(f"   🏫 부서: {metadata.get('department')}")
    print(f"   📂 카테고리: {metadata.get('category')}")
    print(f"   📄 타입: {metadata.get('source_type')}")
    print(f"   🌐 URL: {metadata.get('source_url')}")
    print(f"   📅 날짜: {metadata.get('date')[:10]}")
    print(f"   💬 청크 문장: {doc_text[:200]}...")  # 200자까지만 표시
    print(f"   🧠 벡터 길이: {len(embedding)}")
    print(f"   🔢 벡터 앞 5개 값: {embedding[:5]}")
    print("-" * 70)


📦 전체 저장된 벡터 수: 1095
🔹 ID: Software Department (소프트웨어학과)_공결 규정입니다._chunk_0
   📁 파일명: 공결 규정입니다.
   🏫 부서: Kongju National University Cheonan Campus (공주대학교 천안캠퍼스)/Cheonan Campus Department (천안캠퍼스 학과별 사이트)/Cheonan College of Engineering (천안공과대학)/Software Department (소프트웨어학과)
   📂 카테고리: 커뮤니티 (규정자료실)
   📄 타입: markdown
   🌐 URL: https://sw.kongju.ac.kr/bbs/ZD1180/1426/293859/artclView.do
   📅 날짜: 2025-04-13
   💬 청크 문장: # 공결 규정입니다. **출처:** [https://sw.kongju.ac.kr/bbs/ZD1180/1426/293859/artclView.do](https://sw.kongju.ac.kr/bbs/ZD1180/1426/293859/artclView.do) **작성자:** 임성철 **작성일:** 2024.03.07 ## 본문 <공결 및 병결 관련 국립공주대학...
   🧠 벡터 길이: 384
   🔢 벡터 앞 5개 값: [ 0.04841236  0.03040045  0.10784128  0.02828371 -0.08069456]
----------------------------------------------------------------------
🔹 ID: Software Department (소프트웨어학과)_2022학년도 동계방학 「한국조폐공사」 오픈캠퍼스 현장실습 지원자 모지

In [ ]:
"""
from chromadb import PersistentClient
import unicodedata

# 1. ChromaDB 클라이언트 설정
client = PersistentClient(path="/content/drive/MyDrive/chroma_index")
collection = client.get_collection("knu_cheonan_collection")

# 2. 모든 문서 및 메타데이터 가져오기 (ids 포함)
all_data = collection.get(include=["documents", "metadatas"])

print("🔍 '신입생' 포함된 file_name 검색 중...\n")

found = False

# 3. 순회하면서 '신입생' 포함 여부 확인
for i, meta in enumerate(all_data["metadatas"]):
    file_name = meta.get("file_name", "")
    normalized_name = unicodedata.normalize("NFC", file_name)

    if "신입생 O.T" in normalized_name:
        found = True
        print(f"[{i+1}] ✅ ID: {all_data['ids'][i]}")
        print(f"    📄 file_name: {repr(file_name)}")
        print(f"    📎 metadata: {meta}")
        print(f"    📝 문서 일부: {all_data['documents'][i][:80]}...\n")

if not found:
    print("❌ '신입생'이라는 단어가 포함된 file_name 메타데이터를 찾을 수 없습니다.")
"""

In [ ]:
"""
# 출처 URL 없는거 검색
from chromadb import PersistentClient

# ✅ 1. ChromaDB 컬렉션 열기
client = PersistentClient(path="/content/drive/MyDrive/chroma_index")
collection = client.get_collection(name="knu_cheonan_collection")

# ✅ 2. 전체 문서 조회
results = collection.get(limit=None, include=["documents", "metadatas"])

# ✅ 3. '출처 URL 없음' 필터링
no_url_chunks = []
for id_, doc, meta in zip(results["ids"], results["documents"], results["metadatas"]):
    if meta.get("source_url") == "출처 URL 없음":
        no_url_chunks.append((id_, doc, meta))

# ✅ 4. 결과 출력
print(f"\n🔍 '출처 URL 없음' 청크 개수: {len(no_url_chunks)}")

for id_, doc, meta in no_url_chunks:
    print("\n🆔 ID:", id_)
    print("📝 청크 내용:")
    print(doc)
    print("📎 메타데이터:")
    print(meta)
"""

In [ ]:
# 완전 내부 초기화 ------ 신중히 사용하기 바람 -------
"""
from chromadb import PersistentClient

PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_cheonan_collection"

client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

# 컬렉션 내의 모든 문서 ID를 가져와 삭제
existing_ids = collection.get(limit=None)['ids']
if existing_ids:
    collection.delete(ids=existing_ids)
    print(f"🗑️ 컬렉션 내 모든 데이터 삭제 완료: {len(existing_ids)}개 삭제됨")
else:
    print("✅ 삭제할 데이터가 없습니다. 컬렉션이 이미 비어있습니다.")
"""

In [ ]:
"""
# 컬렉션 Drop
from chromadb import PersistentClient

PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_cheonan_collection"

client = PersistentClient(path=PERSIST_DIR)

# 컬렉션 삭제
client.delete_collection(name=COLLECTION_NAME)
print(f"🗑️ 컬렉션 '{COLLECTION_NAME}' 삭제 완료")
"""

In [ ]:
""" PDF 텍스트 추출 성능 비교 => PyPDF2가 더 잘 나옴
!pip install PyPDF2
!pip install pdfplumber

import PyPDF2
import pdfplumber

with open("/content/drive/MyDrive/document_files/Department (학부)/Cheonan College of Engineering (천안공과대학)/Software Department (소프트웨어학과)/커뮤니티(규정자료실)/미래설계 교과목 이수기준(2021.04.19.).pdf", "rb") as f:
    reader = PyPDF2.PdfReader(f)
    print("[PyPDF2]")
    print(reader.pages[0].extract_text())

with pdfplumber.open("/content/drive/MyDrive/document_files/Department (학부)/Cheonan College of Engineering (천안공과대학)/Software Department (소프트웨어학과)/커뮤니티(규정자료실)/미래설계 교과목 이수기준(2021.04.19.).pdf") as pdf:
    print("\n[pdfplumber]")
    print(pdf.pages[0].extract_text())
"""

In [ ]:
""" 예제 코드에서 사용할 OCR 설치
# Tesseract OCR => 추출이 이상하게 됨
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr
!pip install pytesseract pdf2image Pillow
"""

""" 이거 사용함
# PaddleOCR 설치 (최초 1회) => 한글 지원 매우 좋고, 정확도도 높음, 딥러닝 기반 고정밀 OCR
!pip install paddleocr
!pip install "paddlepaddle==2.6.2" -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!apt-get install -y poppler-utils
!pip install pdf2image
!pip install matplotlib
"""

In [ ]:
"""
from PIL import Image
import pytesseract

# 이미지 경로
IMG_PATH = "/content/drive/MyDrive/sample/공결규정_사본.jpg"

# 이미지 로드 (전처리 없음)
img = Image.open(IMG_PATH)

# OCR 수행
text = pytesseract.image_to_string(img, lang="kor+eng", config="--psm 6 --oem 3")

# 결과 출력
print("🖼️ 이미지 OCR 추출 결과:\n")
print(text)
"""

In [ ]:
"""
from pdf2image import convert_from_path
import pytesseract

# 설정
POPPLER_PATH = "/usr/bin"
PDF_PATH = "/content/drive/MyDrive/sample/2. 홍보자료_ICT 융합인재 양성과정.pdf"
LANG = "kor+eng"

# PDF → 이미지 변환
images = convert_from_path(PDF_PATH, dpi=300, poppler_path=POPPLER_PATH)

# OCR 실행, 이미지 전처리 없어 기본 모델 성능만으로 텍스트 추출
for i, page in enumerate(images):
    text = pytesseract.image_to_string(page, lang=LANG)
    print(f"\n--- Page {i+1} ---")
    print(text)
"""

In [ ]:
"""
from paddleocr import PaddleOCR
from PIL import Image
import numpy as np

# OCR 초기화 (한글 지원)
ocr = PaddleOCR(use_angle_cls=True, lang='korean')

# 이미지 경로
IMG_PATH = "/content/drive/MyDrive/sample/공결규정_사본.jpg"

# 이미지 로드 (전처리 없음)
img = Image.open(IMG_PATH)
image_np = np.array(img.convert("RGB"))[:, :, ::-1]  # PIL → NumPy(BGR)

# OCR 수행
result = ocr.ocr(image_np, cls=True)

# 결과 출력
print("🖼️ 이미지 OCR 추출 결과:\n")
for line in result[0]:
    text = line[1][0]
    print(text)
"""

In [ ]:
"""
from paddleocr import PaddleOCR
from pdf2image import convert_from_path
import numpy as np

# 경로 설정
POPPLER_PATH = "/usr/bin"
PDF_PATH = "/content/drive/MyDrive/sample/2. 홍보자료_ICT 융합인재 양성과정.pdf"

# OCR 초기화 (한글 지원)
ocr = PaddleOCR(use_angle_cls=True, lang='korean')

# PDF → 이미지 변환
images = convert_from_path(PDF_PATH, dpi=300, poppler_path=POPPLER_PATH)

# OCR 실행, 이미지 전처리 없어 기본 모델 성능만으로 텍스트 추출
full_text = ""
for i, page in enumerate(images):
    image_np = np.array(page)[:, :, ::-1]  # PIL → numpy (BGR)

    result = ocr.ocr(image_np, cls=True)

    full_text += f"\n--- Page {i+1} ---\n"
    for line in result[0]:
        full_text += line[1][0] + "\n"

# 결과 출력
print("📄 전체 OCR 텍스트 결과:\n")
print(full_text)
"""

In [ ]:
"""
# 엑셀파일 -> 텍스트 추출
!pip install openpyxl

import pandas as pd

# ✅ 파일 경로 설정 (입력한 그대로 사용)
PDF_PATH = "/content/drive/MyDrive/sample/Software Department (소프트웨어학과).xlsx"

def extract_text_from_excel(file_path):
    try:
        xls = pd.read_excel(file_path, sheet_name=None, dtype=str)
        all_text = ""

        for sheet_name, df in xls.items():
            df = df.fillna("")
            sheet_text = "\n".join(["\t".join(row) for row in df.values.tolist()])
            all_text += f"\n=== 시트: {sheet_name} ===\n{sheet_text}\n"

        return all_text.strip()
    except Exception as e:
        print(f"❌ 엑셀 읽기 실패: {file_path} ({e})")
        return ""

# ✅ 텍스트 추출 실행
extracted_text = extract_text_from_excel(PDF_PATH)

if extracted_text:
    print("✅ 텍스트 추출 완료 (앞부분 미리보기):\n")
    print(extracted_text[:3000])  # 너무 길 경우 앞부분만 미리보기
else:
    print("❌ 텍스트 추출 실패 또는 내용 없음.")
"""